# **LifeHack 2020- Problem #2**

Vision Impairment can be a serious implication on one’s health. According to the key facts stated by the World Health Organization “Globally, at least 2.2 billion people have a vision impairment or blindness, of whom at least 1 billion have a vision impairment that could have been prevented or has yet to be addressed.” Do you have the ability to better detect potential vision issues and help change thousands, if not, millions of lives? 

For example, the given dataset looks at cataract and normal eye image dataset for cataract detection. Participants are encouraged to use other available datasets to better analyse and share their findings regarding the detection of the potential risk of visual problems detection, preferably relating to the Asia region. Indeed this problem statement covers a wide area of visual impairments and does not need to focus solely on cataract. Participants are free to explore other options of eye disease detection.  

  • Present Insight into the data by mean of visualization and/or interactive means
  • Prediction model for eye disease 

For reference:
https://www.kaggle.com/jr2ngb/cataractdataset?Vision 

In [7]:
import os
import numpy as np
import shutil
import matplotlib                  # 2D Plotting Library
import matplotlib.pyplot as plt
import seaborn as sns              # Python Data Visualization Library based on matplotlib
#import geopandas as gpd            # Python Geospatial Data Library
plt.style.use('fivethirtyeight')
%matplotlib inline

In [5]:
# Importing all necessary libraries 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K

Using TensorFlow backend.


In [6]:
img_width, img_height = 2464, 1632

In [14]:
#Creating Train / Val / Test folders 
root_dir = 'cataract_dataset_bundle_archive'
normal = '/1_normal'
cataract = '/2_cataract'
glaucoma = '/2_glaucoma'
retina_disease = '/3_retina_disease'

In [17]:
#Creating Train / Val / Test folders 
root_dir = 'cataract_dataset_bundle_archive'
classes = ['/1_normal', '/2_cataract', '/2_glaucoma', '/3_retina_disease']

for class_type in classes:
    os.makedirs(root_dir + '/train' + class_type)
    os.makedirs(root_dir + '/val' + class_type)
    os.makedirs(root_dir + '/test' + class_type)

    src = root_dir + class_type

    allFiles = os.listdir(src)
    np.random.shuffle(allFiles)
    train_Files, val_Files, test_Files = np.split(np.array(allFiles),
                                                          [int(len(allFiles)*0.7), int(len(allFiles)*0.85)])
                                                          
    train_Files = [src+'/'+ name for name in train_Files.tolist()]
    val_Files = [src+'/' + name for name in val_Files.tolist()]
    test_Files = [src+'/' + name for name in test_Files.tolist()]

    print('Class: ', class_type)
    print('Total images: ', len(allFiles))
    print('Training: ', len(train_Files))
    print('Validation: ', len(val_Files))
    print('Testing: ', len(test_Files))

    for name in train_Files:
        shutil.copy(name, root_dir + "/train" + class_type)
    
    for name in val_Files:
        shutil.copy(name, root_dir + "/val" + class_type)
    
    for name in test_Files:
        shutil.copy(name, root_dir + "/test" + class_type)

Class:  /1_normal
Total images:  300
Training:  210
Validation:  45
Testing:  45
Class:  /2_cataract
Total images:  100
Training:  70
Validation:  15
Testing:  15
Class:  /2_glaucoma
Total images:  101
Training:  70
Validation:  15
Testing:  16
Class:  /3_retina_disease
Total images:  100
Training:  70
Validation:  15
Testing:  15


### Pre-processed data is stored in a new directory - "Data"